In [1]:
%pip install OpenAI

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from openai import OpenAI
import os
import random

client = OpenAI(api_key="API_KEY")

In [14]:
def generate_teacherPrompt(topic, previous):
    try:
        response = client.responses.create(
            model="gpt-4o",
            instructions="You are a professional stable diffusion prompt engineer.",
            input= f"""
                Create one prompt describing an image on the topic of {topic}. 
                The first prompt must be unique (especially main object) from previous prompts [{previous}] and must use simple words suitable for kids and include details about the following categories (Exception: Landscapes image doesn't have a main object) but not too much details: 
                    - Main object: a person, animal, or object
                    - Location: where the main object is
                    - Action: what the main object is doing
                    - Attributes: characteristics of the main object (e.g., color, size, shape)
                    - Background: the environment surrounding the main object (e.g., weather, time of day)

                Output only the two prompts in this format:
                Prompt: [Prompt in one sentence]
                """,
        )

        # Extract and return the generated prompts
        generated_prompts = response.output_text

        return generated_prompts

    except Exception as e:
        return f"An error occurred while refining the prompt: {e}"

teacherPromptList = []

# The list of topics
topics = ["Only Landscapes", "Animals", "Food and Drinks", "Fantasy", "Sci-Fi and Futurastic", "Sports", "Art and Creativity", "Historical and Cultural", "Music and Entertainment"]


for i in range(10):
    promptsSet = generate_teacherPrompt(topics[i%9], teacherPromptList)
    teacherPromptList.append(promptsSet.split("Prompt: ")[1].split("\n")[0])



In [17]:
def generate_studentPrompt(teacher, performance):
    try:
        response = client.responses.create(
            model="gpt-4o",
            instructions="You are a professional stable diffusion prompt engineer.",
            input= f"""
                Create student's prompt describing an image generated by teacher's prompt: {teacher}. 

                The student's prompt use more general or very easy terms while still describing the same image. 
                Quality level of second prompt is {performance}
                Quality Levels:
                - Excellent: 90 ~ 100% simliar to first prompt, it will be similiar length with the first prompt and include all important information (Categories that is in the first prompt).
                - Good: 70 ~ 90% simliar to first prompt, it will be long but loss a little details.
                - Average: 50 ~ 70% simliar to first prompt, it will be little bit short and less detailed.
                - Poor: 30 ~ 50% simliar to first prompt, it will be very short and less detailed or long with wrong information.

                Output only the one prompts in this format:
                Prompt: [Prompt in one sentence]
                """,
        )

        # Extract and return the generated prompts
        generated_prompts = response.output_text

        return generated_prompts

    except Exception as e:
        return f"An error occurred while refining the prompt: {e}"

# The list of performance levels
studentPerformance = ["Excellent", "Good", "Average", "Poor"]
    
# You can change the performance of student to any of the four levels: Excellent (Very similar), Good, Average, Poor (Very distinct)
# You can change the student perforamance have random accuracy using random using random.randint(0, 3)

studentPromptList = []
for i in range(10):
    promptsSet = generate_studentPrompt(teacherPromptList[i], studentPerformance[3])
    studentPromptList.append(promptsSet.split("Prompt: ")[1].split("\n")[0])

In [19]:
import csv
with open('prompts.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Teacher', 'Student'])  # Header
    for t, s in zip(teacherPromptList, studentPromptList):
        writer.writerow([t, s])